<a href="https://colab.research.google.com/github/abhilashhn1993/word_count_with_mapreduce/blob/master/MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import threading
import queue
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

In [3]:
#Reading the Input file
file = open(r"/content/Input.txt")

In [4]:
#Implementing methods to clean function
def cleanText(file):
   words = []
   reg = re.compile("[a-z]+")
   for line in file:
     #token = line.split()
     #words.append(token)
     #for line in words:
     if len(line) > 0:
         words.append(reg.findall(line.lower()))
   words = [i for i in words if i!=[]]
   return words

In [5]:
def unlist(input):
  list = []
  for i in input:
    for word in i:
      list.append(word)
  return list

In [6]:
#Data split function to divide the input data file into two halves
#First part includes the first 5000 lines of the text file and part 2 includes the rest
def split(token):
  words = []
  for word in token:
    words.append(word)
  file1 = unlist(words[0:5000])
  file2 = unlist(words[5001:])
  files = [file1, file2]
  return files

In [7]:
#Mapper function
def mapper1(list, mapper_q1):
  mapping = []
  for word in list:
    mapping.append("%s%s%d"%(word,',',1))
  mapper_q1.put(mapping)

In [8]:
def mapper2(list, mapper_q2):
  mapping = []
  for word in list:
    mapping.append("%s%s%d"%(word,',', 1))
  mapper_q2.put(mapping)

In [9]:
#Function to sort the mapped words
def sort_func(map1, map2):
  map = map1 + map2
  map.sort()
  return map

In [10]:
#Partition function to divide the tokens into two halves
# a-m words and n-z words
def partition(list):
  file1 = []
  file2 = []
  for item in list:
    val = re.search("^[a-m]", item)
    if val!= None:
      file1.append(item)
    else:
      file2.append(item)
  file = [file1,file2]
  return file

In [11]:
#Reducer function
def reducer1(sortedList, reducer_q1):
  word_count = {}
  item = None
  current_count = 0
  for words in sortedList:
    word,count = words.split(',',1)
    count = int(count)
    
    if item == word:
      current_count += count
    else:
      item = word
      current_count = count
    word_count[item] = current_count
  if item == word:
    word_count[item] = current_count
  return reducer_q1.put(word_count)

In [12]:
def reducer2(sortedList, reducer_q2):
  word_count = {}
  item = None
  current_count = 0
  for words in sortedList:
    word,count = words.split(',',1)
    count = int(count)
    
    if item == word:
      current_count += count
    else:
      item = word
      current_count = count
    word_count[item] = current_count
  if item == word:
    word_count[item] = current_count
  return reducer_q2.put(word_count)

In [13]:
#Main method calling the MapReduce function
def MapReduce(path):
  mapper_q1 = queue.Queue()
  mapper_q2 = queue.Queue()
  reducer_q1 = queue.Queue()
  reducer_q2 = queue.Queue()

  wordTokens = cleanText(file)
  filesList = split(wordTokens)

  mapping1 = threading.Thread(target=mapper1, args=(filesList[0], mapper_q1))
  mapping2 = threading.Thread(target=mapper2, args=(filesList[1], mapper_q2))
  mapping1.start()
  mapping2.start()
  mapping1.join()
  mapping2.join()

  mapping1 =[]
  for element in mapper_q1.get():
    mapping1.append(element)

  mapping2 =[]
  for element in mapper_q2.get():
    mapping2.append(element)

  sortedMap = sort_func(mapping1, mapping2)
  sortedFile = partition(sortedMap)

  reducer1 = threading.Thread(target=reducer1, args = (sortedFile[0], reducer_q1))
  reducer2 = threading.Thread(target=reducer2, args = (sortedFile[1], reducer_q2))

  reducer1.start()
  reducer2.start()
  reducer1.join()
  reducer2.join()

  reducer1 = reducer_q1.get()
  reducer2 = reducer_q2.get()

  reducer1.update(reducer2)
  output_file = pd.DataFrame(reducer1.items(), columns = ['Word', 'Count'])
  output_file.to_csv('Output.csv')